<font size=6><b>펀드 운용모델
* 참고: 펀드_운용모델.xlsx

In [1]:
from pykrx import stock
from pykrx import bond
import FinanceDataReader as fdr

from IPython.display import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import time


import warnings
warnings.filterwarnings(action='ignore')

sns.set()
#-------------------- 차트 관련 속성 (한글처리, 그리드) -----------
#plt.rc('font', family='NanumGothicOTF') # For MacOS
plt.rcParams['font.family']= 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False


#-------------------- 주피터 , 출력결과 넓이 늘리기 ---------------
from IPython.core.display import display, HTML
display(HTML("<style>.container{width:100% !important;}</style>"))
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('max_colwidth', None)

# Data Load

## 시총 Top 5 업종명, 업종코드
* LG에너지솔루션, 삼성전자우 제외

In [2]:
start_data='20200101'
end_data='20221229'

df = stock.get_market_cap(end_data) 
top5_ticker = df.sort_values(by='시가총액', ascending=False).index[:7]
top5_name = []
for ticker in top5_ticker:
    names = stock.get_market_ticker_name(ticker)
    top5_name.append(names)
    print(ticker, names)
    time.sleep(1)  
    
top5_ticker = list(top5_ticker)

top5_ticker.remove('373220')
top5_ticker.remove('005935')

top5_name.remove('LG에너지솔루션')
top5_name.remove('삼성전자우')

print(len(top5_ticker), top5_ticker)
print(len(top5_name), top5_name)

005930 삼성전자
373220 LG에너지솔루션
207940 삼성바이오로직스
000660 SK하이닉스
051910 LG화학
005935 삼성전자우
006400 삼성SDI
5 ['005930', '207940', '000660', '051910', '006400']
5 ['삼성전자', '삼성바이오로직스', 'SK하이닉스', 'LG화학', '삼성SDI']


## 종목 기본정보
* 상장주식수, 유동비율
* fnguide 크롤링

In [3]:
from bs4 import BeautifulSoup
import requests
import re

In [4]:
발생주식수_list = []
유동비율_list = []
유동주식수_list = []

for ticker in top5_ticker:
    res = requests.get(f"https://comp.fnguide.com/SVO2/ASP/SVD_main.asp?pGB=1&gicode=A{ticker}&cID=&MenuYn=Y&ReportGB=&NewMenuID=11&stkGb=&strResearchYN=")
    soup = BeautifulSoup(res.text, 'html.parser')
    
    trdata = soup.select("#svdMainGrid1> table > tbody > tr")
    td = trdata[5].select("td")
    
    발행주식수 = td[0].text
    발행주식수 = 발행주식수.split("/")[0].strip()
    발행주식수 = int(''.join(re.findall('[\d.]+', 발행주식수)))
    
    유동비율 = td[1].text
    유동비율 = 유동비율.split('/')[1].strip()
    유동비율= float(유동비율)
    
    유동주식수 = td[1].text
    유동주식수 = 유동주식수.split('/')[0].strip()
    유동주식수 = int(''.join(re.findall('[\d.]+', 유동주식수)))
    
    발생주식수_list.append(발행주식수)
    유동비율_list.append(유동비율)
    유동주식수_list.append(유동주식수)
    
    print(발행주식수, 유동비율, 유동주식수)
    
    time.sleep(1)

5969782550 75.81 4525870769
71174000 24.76 17623664
728002365 73.87 537759511
70592343 66.63 47036786
68764530 73.16 50306427


In [5]:
발생주식수_list

[5969782550, 71174000, 728002365, 70592343, 68764530]

In [6]:
유동비율_list

[75.81, 24.76, 73.87, 66.63, 73.16]

In [7]:
유동주식수_list  

[4525870769, 17623664, 537759511, 47036786, 50306427]

## 일자별 주가

In [8]:
주가_list = []
for ticker in top5_ticker:
    names = stock.get_market_ticker_name(ticker)
    print(ticker)
    closev = stock.get_market_ohlcv(start_data, end_data, ticker)[['종가']]
    closev.columns = [names]
    주가_list.append(closev)
    time.sleep(1)  
    
주가_df = pd.concat(주가_list, axis=1)  
주가_df.head()

005930


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [39]:
주가_df.to_csv('./datasets/일자별주가.csv')

* <font color=red><b>API 막혀있을 경우 아래 코드로 대체

In [40]:
주가_df = pd.read_csv("./datasets/일자별주가.csv",  parse_dates=["날짜"])
주가_df = 주가_df.set_index("날짜")
주가_df.head()

,Unnamed: 0.1,Unnamed: 0,삼성전자,삼성바이오로직스,SK하이닉스,LG화학,삼성SDI
날짜,,,,,,,
2020-01-02,0,0,55200,423485,94700,314000,232000
2020-01-03,1,1,55500,417061,94500,311000,229000
2020-01-06,2,2,55500,407178,94300,307000,230000
2020-01-07,3,3,55800,414096,94000,311000,231500
2020-01-08,4,4,56800,403719,97400,302000,229000


# 일자별 시가 총액

In [41]:
# 유동주식수 = np.array(발생주식수_list) * np.array(유동비율_list)*0.01
# 시가총액_df = 주가_df*유동주식수
시가총액_df = 주가_df*np.array(유동주식수_list)
시가총액_df.head()

ValueError: Unable to coerce to Series, length must be 7: given 5

In [ ]:
# 시가총액_df['합계'] = 시가총액_df.iloc[:,0]+시가총액_df.iloc[:,1]+시가총액_df.iloc[:,2]+시가총액_df.iloc[:,3]+시가총액_df.iloc[:,4]
시가총액_df['합계'] = 시가총액_df.sum(axis=1)
시가총액_df.head()

In [37]:
시가총액_df['지수'] = 시가총액_df['합계']/시가총액_df.iloc[0,5]
시가총액_df.head()

NameError: name '시가총액_df' is not defined

# 펀드

## 편입비중

In [38]:
편입비중_df = 시가총액_df[top5_name]
편입비중_df = 편입비중_df.div(시가총액_df['합계'], axis=0)
편입비중_df.head()

NameError: name '시가총액_df' is not defined

## 설정좌수, 환매좌수

In [31]:
좌수 = pd.read_csv('./datasets/설정환매_랜덤.csv', encoding = 'utf-8')
좌수.columns = ['날짜','설정','환매']
좌수['날짜'] = 좌수['날짜'].astype('datetime64')
좌수 = 좌수.set_index('날짜')
좌수.head()

,설정,환매
날짜,,
2020-01-02,1000000,0
2020-01-03,50000,0
2020-01-06,150000,100000
2020-01-07,100000,100000
2020-01-08,150000,100000


In [32]:
좌수.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 742 entries, 2020-01-02 to 2022-12-29
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   설정      742 non-null    int64
 1   환매      742 non-null    int64
dtypes: int64(2)
memory usage: 17.4 KB


In [33]:
좌수['총좌수'] = (좌수['설정']-좌수['환매']).cumsum()
좌수.head()

,설정,환매,총좌수
날짜,,,
2020-01-02,1000000,0,1000000
2020-01-03,50000,0,1050000
2020-01-06,150000,100000,1100000
2020-01-07,100000,100000,1100000
2020-01-08,150000,100000,1150000


## iNAV

In [34]:
기준가 = 10000
iNAV = (시가총액_df['지수']/시가총액_df['지수'][0])*기준가
# iNAV = iNAV.astype(int)
iNAV

NameError: name '시가총액_df' is not defined

## AUM

In [35]:
AUM = iNAV*좌수['총좌수']
AUM

NameError: name 'iNAV' is not defined

## 일자별 매매내역

In [ ]:
주식잔고_df = 편입비중_df[top5_name]

주식잔고_df = 주식잔고_df.mul(AUM, axis=0)
주식잔고_df = 주식잔고_df.div(주가_df)

주식잔고_df = 주식잔고_df.astype(int)

주식잔고_df.head()

In [ ]:
매매내역_df = 주식잔고_df.diff()
매매내역_df.iloc[0,:] = 주식잔고_df.iloc[0,:]
매매내역_df.head()

## Total Fund Result

In [ ]:
fdf = pd.concat([편입비중_df,시가총액_df['지수'],  좌수, iNAV, AUM, 매매내역_df, 주식잔고_df], axis=1)
fdf.head()

In [ ]:
fdf.columns = ['편입비중_삼성전자', '편입비중_삼성바이오로직스', '편입비중_SK하이닉스', '편입비중_LG화학', '편입비중_삼성SDI',
              '지수', '설정좌수', '환매좌수', '총좌수', 'iNAV', 'AUM',
              '매매내역_삼성전자', '매매내역_삼성바이오로직스', '매매내역_SK하이닉스', '매매내역_LG화학', '매매내역_삼성SDI',
              '주식잔고_삼성전자', '주식잔고_삼성바이오로직스', '주식잔고_SK하이닉스', '주식잔고_LG화학', '주식잔고_삼성SDI']
fdf.head()

In [ ]:
fdf.tail()

In [ ]:
fdf.to_csv("실습07_펀드_운용모델_최종.csv")

# 수익률 비교

In [16]:
fdf = pd.read_csv("실습07_펀드_운용모델_최종.csv", parse_dates=['날짜'])
fdf = fdf.set_index("날짜")
fdf.head()

FileNotFoundError: [Errno 2] No such file or directory: '실습07_펀드_운용모델_최종.csv'

## KOSPI KOSPI 100 
* ref : https://github.com/sharebook-kr/pykrx
<pre>
for ticker in stock.get_index_ticker_list():
    print(ticker, stock.get_index_ticker_name(ticker))
1001 코스피
1028 코스피 200
1034 코스피 100

In [17]:
# 인덱스 구성종목 조회
pdf = stock.get_index_portfolio_deposit_file("1001")
print(len(pdf), pdf[:3])

824 ['005930', '373220', '000660']


In [18]:
# # 인덱스 OHLCV
# kospi_df = stock.get_index_ohlcv("20200101", "20221231", "1001")
# kospi_df.head()

In [19]:
import yfinance as yf
# tickers      = ['^GSPC', '^KS11','IEF', '148070.KS'] # 주식:지수,  채권:10년물 국채 
tickers      = ['^KS11','148070.KS'] # 주식:지수,  채권:10년물 국채 
start_date = '2020-01-01'
end_date = '2022-12-31'

sample=pd.DataFrame()
for i, ticker in enumerate(tickers):
    ticker = yf.Ticker(ticker)
    s = ticker.history(interval='1d', start=start_date, end=end_date, auto_adjust=False)[['Close']]
    s.index = pd.to_datetime(s.index.strftime('%Y-%m-%d'), format='%Y-%m-%d')
    sample = pd.concat([sample, s], axis=1)
    
# sample.columns = ['S&P500', 'KOSPI', 'US-BOND10Y', 'KR-BOND10Y']
sample.columns = ['KOSPI', 'BOND10Y']
sample.head()

,KOSPI,BOND10Y
Date,,
2020-01-02,2175.169922,123285.0
2020-01-03,2176.459961,124030.0
2020-01-06,2155.070068,124360.0
2020-01-07,2175.540039,123590.0
2020-01-08,2151.310059,123360.0


In [20]:
sample['KOSPI지수'] = sample['KOSPI'] / sample.iloc[0]['KOSPI']
sample['BOND10지수'] = sample['BOND10Y'] / sample.iloc[0]['BOND10Y']
sample.head()

,KOSPI,BOND10Y,KOSPI지수,BOND10지수
Date,,,,
2020-01-02,2175.169922,123285.0,1.000000,1.000000
2020-01-03,2176.459961,124030.0,1.000593,1.006043
2020-01-06,2155.070068,124360.0,0.990759,1.008720
2020-01-07,2175.540039,123590.0,1.000170,1.002474
2020-01-08,2151.310059,123360.0,0.989031,1.000608


In [21]:
fdf.shape, sample.shape

NameError: name 'fdf' is not defined

In [ ]:
# fdf 날짜 sampe: Date
tot = pd.merge(sample[['KOSPI지수','BOND10지수']], fdf["지수"], left_index=True, right_index=True)
tot.columns = ['KOSPI지수','BOND10지수','아시아5지수']
tot.head()

In [ ]:
# pd.options.plotting.backend = "plotly"   
tot.plot(color=['orange','blue','red'], figsize=(10,4))
plt.show()

## 일간수익률